In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools
from operator import itemgetter

import holoviews as hv
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.models import TapTool
from bokeh.models import CustomJS
from bokeh.io import show, curdoc
from bokeh.plotting import figure
import webbrowser

from holoviews import streams

from scipy.ndimage import median_filter, gaussian_filter
from pathlib import Path
import sys
import traceback
import json
import datetime
import subprocess
from collections import Counter
import pickle
import os
import platform

import cv2
from moviepy.editor import VideoClip
from moviepy.editor import VideoFileClip
from moviepy.video.fx import all as vfx
import pygame

In [ ]:
# Get a video

Video = Path("/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230721_Feedingstate_4_PM_Videos_Tracked/arena5/corridor5/corridor5.mp4")

Tracking = Path("/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230721_Feedingstate_4_PM_Videos_Tracked/arena5/corridor5/corridor5_tracked_fly_full.000_corridor5.analysis.h5")

In [ ]:
f = h5py.File(Tracking.as_posix(), "r")

In [ ]:
list(f.keys())

In [ ]:
keypoints = f["node_names"]

keypoints[:]

In [ ]:
def extract_coordinates(h5_file):
    """
    Extracts the x and y coordinates from a h5 file. Only works for single object tracking. For skeleton tracking,
    use extract_skeleton.

    Args:
        h5_file (str): The path to the h5 file.

    Returns:
        tuple: Two np.ndarrays representing the x and y coordinates.
    """

    with h5py.File(h5_file, "r") as f:
        locs = f["tracks"][:].T
        y = locs[:, :, 1, :].squeeze()
        x = locs[:, :, 0, :].squeeze()
        keypoints = f["keypoints"][:].T
        
    return x, y, keypoints

In [ ]:
# Load the tracking data

tracks = extract_coordinates(Tracking)
    

the shape s like this: x, y . Below is : frame. Below is : keypoint.

so tracks [0] [0] is x for frame 1.

In [ ]:
def generate_clip(video, tracks, start_frame, end_frame, outpath="output.avi"):
    cap = cv2.VideoCapture(str(video))

    # Get the frame rate
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Get the width and height of frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(outpath, fourcc, fps, (width, height))

    try:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        for i in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break

            # Get the current tracking data
            x = tracks[0][i]
            y = tracks[1][i]

            # Draw the tracking data on the frame
            for j in range(len(x)):
                if not np.isnan(x[j]) and not np.isnan(y[j]):
                    cv2.circle(frame, (int(x[j]), int(y[j])), 5, (0, 255, 0), -1)

            # Write the frame to the output video
            out.write(frame)
    finally:
        # Release the video capture and writer objects
        cap.release()
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

In [ ]:
generate_clip(Video, tracks, 1000, int(1020*29), "output.avi")